In [9]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
import jieba
import re
from sklearn.ensemble import RandomForestClassifier
import math
import gc
import warnings

warnings.filterwarnings('ignore')

def cutwordtostr(sent):
    jieba.load_userdict("C:/Users/Administrator.SC-201905252025/Desktop/test/qg_keywords.txt")
    wordList=jieba.lcut(sent,cut_all=True)
    words = list(set(wordList))
    return [words,' '.join(words)]
    del wordList
    del words
    gc.collect()

def key_count(words):
    keyword = [line.strip() for line in open("C:/Users/Administrator.SC-201905252025/Desktop/test/qg_keywords.txt", 'r', encoding="UTF-8").readlines()]
    count_dict={}
    for i in keyword:
        a=words.count(i)
        if a>0:
            count_dict[i]=a
    return [len(count_dict),sum(count_dict.values())]
    del count_dict
    del a
    del keyword
    gc.collect()

def vectorize(df):
    count_vect = TfidfVectorizer(binary=False,decode_error='ignore')
    word_vec = count_vect.fit_transform(df['cutwordstr'])
    df['vec'] = word_vec.todense().tolist()
    return df
    del count_vect
    del word_vec
    gc.collect()

def process(df):
    df['result'] = df['content'].apply(cutwordtostr)
    df['cutword'] = df['result'].apply(lambda x:x[0])
    df['cutwordstr'] = df['result'].apply(lambda x:x[1])

    df['result'] = df['cutword'].apply(key_count)
    df['word_count'] = df['result'].apply(lambda x:x[0])
    df['key_count'] = df['result'].apply(lambda x:x[1])

    df.drop(['result'], axis=1, inplace=True)
    df = vectorize(df)
    df=df.sort_values(by=['key_count'])

    df_positive = df.loc[(df.key_count>2) & (df.word_count>1)]
    df_positive['label'] = 0
    df_negative = df.loc[df.key_count<2,].iloc[0:df_positive.shape[0]]
    df_negative['label'] = 1
    df_train = pd.concat([df_positive,df_negative])

    # 随机森林
    model = RandomForestClassifier()
    model.fit(np.array(df_train['vec']).tolist(), np.array(df_train['label']))
    df['label'] = model.predict(np.array(df['vec']).tolist())

    df.drop(['key_count', 'word_count', 'cutword', 'cutwordstr', 'vec'], axis=1, inplace = True)
    df.to_csv("C:/Users/Administrator.SC-201905252025/Desktop/RF_union.csv", mode='a', header=False, index=False)
    del df
    del df_positive
    del df_negative
    del df_train
    del model
    gc.collect()


if __name__ == "__main__":
    file = pd.read_excel("C:/Users/Administrator.SC-201905252025/Desktop/test/groupchat.xlsx", sheet_name='Sheet1', names=['authaccount','from_id', 'to_id', 'content'])
    df = pd.DataFrame(file)
    m = df.shape[0]  
    mini_batch_size = 5000
    num_complete_minibatches = math.floor(m/mini_batch_size)
    
#     for k in range(0, num_complete_minibatches):
    for k in range(0, 3):
        process(df.iloc[k * mini_batch_size : (k + 1) * mini_batch_size, :])
        
#     if m % mini_batch_size != 0:
#         process(df.iloc[num_complete_minibatches * mini_batch_size : , :])

In [1]:
import pandas as pd
import numpy as np

file = pd.read_excel("C:/Users/Administrator.SC-201905252025/Desktop/test/groupchat.xlsx", sheet_name='Sheet1', names=['authaccount','from_id', 'to_id', 'content'])
df = pd.DataFrame(file)
m = df.shape[0] 

permutation = list(np.random.permutation(m))
df = df.iloc[permutation,:].iloc[0:200,:]

writer = pd.ExcelWriter("C:/Users/Administrator.SC-201905252025/Desktop/test.xlsx", engine='xlsxwriter',options={'strings_to_urls':False})
writer.book.strings_to_urls = False
df.to_excel(writer, 'Sheet1', index='False')
writer.save()
print(1)

1


In [21]:
import pandas as pd
import numpy as np
import math
from numpy import array
 
file = pd.read_excel("C:/Users/Administrator.SC-201905252025/Desktop/terrorunion.xlsx", sheet_name='Sheet1', names=['手机号','网络搜索次数', 'VPN使用次数', '敏感聊天软件使用次数','网络发言次数'])
df = pd.DataFrame(file)
# df.dropna()
df = df.fillna(value=0)
df.drop(['手机号'],axis=1, inplace=True)
# df['网络搜索次数'] = 3*df['网络搜索次数']
# df['VPN使用次数'] = 1*df['VPN使用次数']
# df['敏感聊天软件使用次数'] = 2*df['敏感聊天软件使用次数']
# df['网络发言次数'] = 3*df['网络发言次数']
 
def cal_weight(data):
    data = (data - data.min())/(data.max() - data.min())
    m,n=data.shape
    data=data.as_matrix(columns=None)
    k=1/np.log(m)
    print(data)
    yij=data.sum(axis=0)
    print(yij)
    pij=data/yij
#     print(pij)
    test=pij*np.log(pij)
#     print(test)
    test=np.nan_to_num(test)
#     print(test)
    ej=-k*(test.sum(axis=0))
    wi=(1-ej)/np.sum(1-ej) 
    print(wi)
 
 
if __name__ == '__main__':
    cal_weight(df) 

[[0.04       0.         1.         0.        ]
 [0.         0.25       0.         0.05882353]
 [1.         0.         0.         0.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         0.58823529]
 [0.36       0.         0.         0.        ]
 [0.         0.         0.         0.41176471]
 [0.         0.         0.         0.35294118]
 [0.         0.         0.         0.35294118]
 [0.         0.         0.         0.29411765]
 [0.16       0.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         0.         0.44444444 0.        ]
 [0.12       0.         0.         0.        ]
 [0.         0.         0.         0.17647059]
 [0.08       0.         0.         0.        ]
 [0.08       0.         0.         0.        ]
 [0.         0.5        0.         0.        ]
 [0.         0.5        0.         0.        ]
 [0.         0.         0.22222222 0.        ]
 [0.         0.         0.         0.11764706]
 [0.         

d:\python35\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
d:\python35\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
d:\python35\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in multiply
